<a href="https://colab.research.google.com/github/slin35/RobotProducer/blob/main/lab1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!unzip data.zip

Archive:  data.zip
  inflating: tmdb_5000_credits.csv   
  inflating: tmdb_5000_movies.csv    
  inflating: uncased_L-12_H-768_A-12/bert_config.json  
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001  
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.index  
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.meta  
  inflating: uncased_L-12_H-768_A-12/vocab.txt  


In [5]:
!pip install transformers
!pip install bert-tensorflow==1.0.1

     |████████████████████████████████| 2.1MB 4.5MB/s 
     |████████████████████████████████| 870kB 34.3MB/s 
     |████████████████████████████████| 3.3MB 32.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.44-cp37-none-any.whl size=886084 sha256=246b265c4c48d2bc4bef4606d7b3e0aa45d7437cda5c7d2780e114b280c66381
  Stored in directory: /root/.cache/pip/wheels/3e/fb/c0/13ab4d63d537658f448366744654323077c4d90069b6512f3c
Successfully built sacremoses
     |████████████████████████████████| 71kB 3.5MB/s 


In [6]:
%tensorflow_version 1.x
import tensorflow as tf
import tensorflow_hub as hub

from transformers import pipeline

import pandas as pd
import json
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
import numpy as np
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
import os
import collections
from datetime import datetime


import bert

from bert import run_classifier
from bert import optimization
from bert import tokenization
from bert import modeling
from bert import run_classifier_with_tfhub

TensorFlow 1.x selected.



In [7]:


# predict the title of the movie
def get_title(overview: str):

  question_answerer = pipeline('question-answering')
  answer = question_answerer({
    'question': 'What is the name of the movie ?',
    'context': overview
  })

  return answer['answer']


In [8]:
def get_data():
  # import data
  data1 = pd.read_csv('./tmdb_5000_movies.csv')
  data2 = pd.read_csv('./tmdb_5000_credits.csv')

  data1 = data1[['genres', 'title', 'overview', 'id']]
  data2 = data2[['movie_id', 'title', 'cast', 'crew']]

  data = pd.merge(data1, data2, left_on='id', right_on='movie_id', how='inner')
  data = data[['id', 'title_x', 'overview', 'genres', 'cast', 'crew']]
  data.rename({'title_x': 'title'}, axis=1, inplace=True)

  # cleanup genres
  data['genres'] = [json.loads(i) if i != [] else [] for i in data['genres']]
  data['genres'] = [[j['name'] if 'name' in j else [] for j in i] for i in data['genres']]

  # cleanup cast
  data['cast'] = [json.loads(i) if i != [] else [] for i in data['cast']]
  data['cast'] = [[j['name'] if 'name' in j else [] for j in i] for i in data['cast']]

  # cleanup crew
  data['crew'] = [json.loads(i) if i != [] else [] for i in data['crew']]
  data['crew'] = [[j['name']  for j in i if 'job' in j and j['job'] == 'Director'] for i in data['crew']]

  # get rid of rows with empty values in overview, genres, cast, crew
  data.replace("", float("NaN"), inplace=True)
  data.dropna(inplace=True)
  data = data[~data.genres.str.len().eq(0)]
  data = data[~data.cast.str.len().eq(0)]
  data = data[~data.crew.str.len().eq(0)]

  return data

In [9]:
def get_unique(data: pd.DataFrame, label: str):
  result = []
  [[result.append(item) for item in row if item not in result] for row in data[label]]
  return result

In [10]:


# BERT-based, uncased model: 12-layer, 768-hidden, 12-heads, 110M parematers
BERT_VOCAB= './uncased_L-12_H-768_A-12/vocab.txt'
BERT_INIT_CHKPNT = './uncased_L-12_H-768_A-12/bert_model.ckpt'
BERT_CONFIG = './uncased_L-12_H-768_A-12/bert_config.json'


# BERT-large, uncased model: 24-layer, 1024-hidden, 16-heads, 340M parameters
#BERT_VOCAB= './uncased_L-24_H-1024_A-16/vocab.txt'
#BERT_INIT_CHKPNT = './uncased_L-24_H-1024_A-16/bert_model.ckpt'
#BERT_CONFIG = './uncased_L-24_H-1024_A-16/bert_config.json'

def get_tokenizer():
  tokenization.validate_case_matches_checkpoint(True,BERT_INIT_CHKPNT)
  return tokenization.FullTokenizer(vocab_file=BERT_VOCAB, do_lower_case=True)
  

# InputExample for object with multiple labels
class InputExample(object):
  def __init__(self, guid, text_a, text_b=None, labels=None):
    self.guid = guid
    self.text_a = text_a
    self.text_b = text_b
    self.labels = labels

# InputFeature for object with multiple labels
class InputFeatures(object):
  def __init__(self, input_ids, input_mask, segment_ids, label_ids, is_real_example=True):
    self.input_ids = input_ids
    self.input_mask = input_mask
    self.segment_ids = segment_ids
    self.label_ids = label_ids
    self.is_real_example = is_real_example

# create InputExamples
def create_examples(df: pd.DataFrame, list_of_train_labels: list, num_labels: int, set_labels=True):
  examples = []
  
  for index, row in df.iterrows():
    guid = row['id']
    text_a = row['overview']
    if set_labels:
      labels = list_of_train_labels[index]
    else:
      labels = [0] * num_labels
    examples.append(InputExample(guid=guid, text_a=text_a, labels=labels))
  return examples

# for a single sequence convert_example_to_features
def convert_examples_to_features(examples, max_seq_length, tokenizer):
  features = []

  for (idx, example) in enumerate(examples):

    tokens_a = tokenizer.tokenize(example.text_a)
    tokens_b = None;

    if len(tokens_a) > max_seq_length - 2:
      tokens_a = tokens_a[:(max_seq_length - 2)]

    tokens = ["[CLS]"] + tokens_a + ["[SEP]"]
    segment_ids = [0] * len(tokens)           # its a single sequence, so segment_ids are all 0

    input_ids = tokenizer.convert_tokens_to_ids(tokens)

    input_mask = [1] * len(input_ids)

    # padding up to sequence length
    padding = [0] * (max_seq_length - len(input_ids))
    input_ids += padding
    input_mask += padding 
    segment_ids += padding 

    assert len(input_ids) == max_seq_length
    assert len(input_mask) == max_seq_length
    assert len(segment_ids) == max_seq_length

    labels_ids = []
    
    for label in example.labels:
        labels_ids.append(int(label))

    feature = InputFeatures(input_ids, input_mask, segment_ids, labels_ids)
    features.append(feature)

  return features


In [11]:
def create_model(bert_config, is_training, input_ids, input_mask, segment_ids,
                 labels, num_labels, use_one_hot_embeddings):
    """Creates a classification model."""
    model = modeling.BertModel(
        config=bert_config,
        is_training=is_training,
        input_ids=input_ids,
        input_mask=input_mask,
        token_type_ids=segment_ids,
        use_one_hot_embeddings=use_one_hot_embeddings)

    # In the demo, we are doing a simple classification task on the entire
    # segment.
    #
    # If you want to use the token-level output, use model.get_sequence_output()
    # instead.
    output_layer = model.get_pooled_output()

    hidden_size = output_layer.shape[-1].value

    output_weights = tf.get_variable(
        "output_weights", [num_labels, hidden_size],
        initializer=tf.truncated_normal_initializer(stddev=0.02))

    output_bias = tf.get_variable(
        "output_bias", [num_labels], initializer=tf.zeros_initializer())

    with tf.variable_scope("loss"):
        if is_training:
            # I.e., 0.1 dropout
            output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)

        logits = tf.matmul(output_layer, output_weights, transpose_b=True)
        logits = tf.nn.bias_add(logits, output_bias)
        
        # probabilities = tf.nn.softmax(logits, axis=-1) ### multiclass case
        probabilities = tf.nn.sigmoid(logits)#### multi-label case
        
        labels = tf.cast(labels, tf.float32)
        tf.logging.info("num_labels:{};logits:{};labels:{}".format(num_labels, logits, labels))
        per_example_loss = tf.nn.sigmoid_cross_entropy_with_logits(labels=labels, logits=logits)
        loss = tf.reduce_mean(per_example_loss)

        # probabilities = tf.nn.softmax(logits, axis=-1)
        # log_probs = tf.nn.log_softmax(logits, axis=-1)
        #
        # one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)
        #
        # per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
        # loss = tf.reduce_mean(per_example_loss)

        return (loss, per_example_loss, logits, probabilities)


def model_fn_builder(bert_config, num_labels, init_checkpoint, learning_rate,
                     num_train_steps, num_warmup_steps, use_tpu,
                     use_one_hot_embeddings):
    """Returns `model_fn` closure for TPUEstimator."""

    def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
        """The `model_fn` for TPUEstimator."""

        #tf.logging.info("*** Features ***")
        #for name in sorted(features.keys()):
        #    tf.logging.info("  name = %s, shape = %s" % (name, features[name].shape))

        input_ids = features["input_ids"]
        input_mask = features["input_mask"]
        segment_ids = features["segment_ids"]
        label_ids = features["label_ids"]
        is_real_example = None
        if "is_real_example" in features:
             is_real_example = tf.cast(features["is_real_example"], dtype=tf.float32)
        else:
             is_real_example = tf.ones(tf.shape(label_ids), dtype=tf.float32)

        is_training = (mode == tf.estimator.ModeKeys.TRAIN)

        (total_loss, per_example_loss, logits, probabilities) = create_model(
            bert_config, is_training, input_ids, input_mask, segment_ids, label_ids,
            num_labels, use_one_hot_embeddings)

        tvars = tf.trainable_variables()
        initialized_variable_names = {}
        scaffold_fn = None
        if init_checkpoint:
            (assignment_map, initialized_variable_names
             ) = modeling.get_assignment_map_from_checkpoint(tvars, init_checkpoint)
            if use_tpu:

                def tpu_scaffold():
                    tf.train.init_from_checkpoint(init_checkpoint, assignment_map)
                    return tf.train.Scaffold()

                scaffold_fn = tpu_scaffold
            else:
                tf.train.init_from_checkpoint(init_checkpoint, assignment_map)

        tf.logging.info("**** Trainable Variables ****")
        for var in tvars:
            init_string = ""
            if var.name in initialized_variable_names:
                init_string = ", *INIT_FROM_CKPT*"
            #tf.logging.info("  name = %s, shape = %s%s", var.name, var.shape,init_string)

        output_spec = None
        if mode == tf.estimator.ModeKeys.TRAIN:

            train_op = optimization.create_optimizer(
                total_loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu)

            output_spec = tf.estimator.EstimatorSpec(
                mode=mode,
                loss=total_loss,
                train_op=train_op,
                scaffold=scaffold_fn)
        elif mode == tf.estimator.ModeKeys.EVAL:

            def metric_fn(per_example_loss, label_ids, probabilities, is_real_example):

                logits_split = tf.split(probabilities, num_labels, axis=-1)
                label_ids_split = tf.split(label_ids, num_labels, axis=-1)
                # metrics change to auc of every class
                eval_dict = {}
                for j, logits in enumerate(logits_split):
                    label_id_ = tf.cast(label_ids_split[j], dtype=tf.int32)
                    current_auc, update_op_auc = tf.metrics.auc(label_id_, logits)
                    eval_dict[str(j)] = (current_auc, update_op_auc)
                eval_dict['eval_loss'] = tf.metrics.mean(values=per_example_loss)
                return eval_dict

                ## original eval metrics
                # predictions = tf.argmax(logits, axis=-1, output_type=tf.int32)
                # accuracy = tf.metrics.accuracy(
                #     labels=label_ids, predictions=predictions, weights=is_real_example)
                # loss = tf.metrics.mean(values=per_example_loss, weights=is_real_example)
                # return {
                #     "eval_accuracy": accuracy,
                #     "eval_loss": loss,
                # }

            eval_metrics = metric_fn(per_example_loss, label_ids, probabilities, is_real_example)
            output_spec = tf.estimator.EstimatorSpec(
                mode=mode,
                loss=total_loss,
                eval_metric_ops=eval_metrics,
                scaffold=scaffold_fn)
        else:
            print("mode:", mode,"probabilities:", probabilities)
            output_spec = tf.estimator.EstimatorSpec(
                mode=mode,
                predictions={"probabilities": probabilities},
                scaffold=scaffold_fn)
        return output_spec

    return model_fn

In [12]:
class PaddingInputExample(object):
    """Fake example so the num input examples is a multiple of the batch size.
    When running eval/predict on the TPU, we need to pad the number of examples
    to be a multiple of the batch size, because the TPU requires a fixed batch
    size. The alternative is to drop the last batch, which is bad because it means
    the entire output data won't be generated.
    We use this class instead of `None` because treating `None` as padding
    battches could cause silent errors.
    """
    
    
def convert_single_example(ex_index, example, max_seq_length,
                           tokenizer):
    """Converts a single `InputExample` into a single `InputFeatures`."""

    if isinstance(example, PaddingInputExample):
        return InputFeatures(
            input_ids=[0] * max_seq_length,
            input_mask=[0] * max_seq_length,
            segment_ids=[0] * max_seq_length,
            label_ids=0,
            is_real_example=False)

    tokens_a = tokenizer.tokenize(example.text_a)
    tokens_b = None
    if example.text_b:
        tokens_b = tokenizer.tokenize(example.text_b)

    if tokens_b:
        # Modifies `tokens_a` and `tokens_b` in place so that the total
        # length is less than the specified length.
        # Account for [CLS], [SEP], [SEP] with "- 3"
        _truncate_seq_pair(tokens_a, tokens_b, max_seq_length - 3)
    else:
        # Account for [CLS] and [SEP] with "- 2"
        if len(tokens_a) > max_seq_length - 2:
            tokens_a = tokens_a[0:(max_seq_length - 2)]

    # The convention in BERT is:
    # (a) For sequence pairs:
    #  tokens:   [CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]
    #  type_ids: 0     0  0    0    0     0       0 0     1  1  1  1   1 1
    # (b) For single sequences:
    #  tokens:   [CLS] the dog is hairy . [SEP]
    #  type_ids: 0     0   0   0  0     0 0
    #
    # Where "type_ids" are used to indicate whether this is the first
    # sequence or the second sequence. The embedding vectors for `type=0` and
    # `type=1` were learned during pre-training and are added to the wordpiece
    # embedding vector (and position vector). This is not *strictly* necessary
    # since the [SEP] token unambiguously separates the sequences, but it makes
    # it easier for the model to learn the concept of sequences.
    #
    # For classification tasks, the first vector (corresponding to [CLS]) is
    # used as the "sentence vector". Note that this only makes sense because
    # the entire model is fine-tuned.
    tokens = []
    segment_ids = []
    tokens.append("[CLS]")
    segment_ids.append(0)
    for token in tokens_a:
        tokens.append(token)
        segment_ids.append(0)
    tokens.append("[SEP]")
    segment_ids.append(0)

    if tokens_b:
        for token in tokens_b:
            tokens.append(token)
            segment_ids.append(1)
        tokens.append("[SEP]")
        segment_ids.append(1)

    input_ids = tokenizer.convert_tokens_to_ids(tokens)

    # The mask has 1 for real tokens and 0 for padding tokens. Only real
    # tokens are attended to.
    input_mask = [1] * len(input_ids)

    # Zero-pad up to the sequence length.
    while len(input_ids) < max_seq_length:
        input_ids.append(0)
        input_mask.append(0)
        segment_ids.append(0)

    assert len(input_ids) == max_seq_length
    assert len(input_mask) == max_seq_length
    assert len(segment_ids) == max_seq_length

    labels_ids = []
    for label in example.labels:
        labels_ids.append(int(label))


    feature = InputFeatures(
        input_ids=input_ids,
        input_mask=input_mask,
        segment_ids=segment_ids,
        label_ids=labels_ids,
        is_real_example=True)
    return feature


def file_based_convert_examples_to_features(
        examples, max_seq_length, tokenizer, output_file):
    """Convert a set of `InputExample`s to a TFRecord file."""

    writer = tf.python_io.TFRecordWriter(output_file)

    for (ex_index, example) in enumerate(examples):
        #if ex_index % 10000 == 0:
            #tf.logging.info("Writing example %d of %d" % (ex_index, len(examples)))

        feature = convert_single_example(ex_index, example,
                                         max_seq_length, tokenizer)

        def create_int_feature(values):
            f = tf.train.Feature(int64_list=tf.train.Int64List(value=list(values)))
            return f

        features = collections.OrderedDict()
        features["input_ids"] = create_int_feature(feature.input_ids)
        features["input_mask"] = create_int_feature(feature.input_mask)
        features["segment_ids"] = create_int_feature(feature.segment_ids)
        features["is_real_example"] = create_int_feature(
            [int(feature.is_real_example)])
        if isinstance(feature.label_ids, list):
            label_ids = feature.label_ids
        else:
            label_ids = feature.label_ids[0]
        features["label_ids"] = create_int_feature(label_ids)

        tf_example = tf.train.Example(features=tf.train.Features(feature=features))
        writer.write(tf_example.SerializeToString())
    writer.close()


def file_based_input_fn_builder(input_file, seq_length, is_training,
                                drop_remainder, num_labels):
    """Creates an `input_fn` closure to be passed to TPUEstimator."""

    name_to_features = {
        "input_ids": tf.FixedLenFeature([seq_length], tf.int64),
        "input_mask": tf.FixedLenFeature([seq_length], tf.int64),
        "segment_ids": tf.FixedLenFeature([seq_length], tf.int64),
        "label_ids": tf.FixedLenFeature([num_labels], tf.int64),
        "is_real_example": tf.FixedLenFeature([], tf.int64),
    }

    def _decode_record(record, name_to_features):
        """Decodes a record to a TensorFlow example."""
        example = tf.parse_single_example(record, name_to_features)

        # tf.Example only supports tf.int64, but the TPU only supports tf.int32.
        # So cast all int64 to int32.
        for name in list(example.keys()):
            t = example[name]
            if t.dtype == tf.int64:
                t = tf.to_int32(t)
            example[name] = t

        return example

    def input_fn(params):
        """The actual input function."""
        batch_size = params["batch_size"]

        # For training, we want a lot of parallel reading and shuffling.
        # For eval, we want no shuffling and parallel reading doesn't matter.
        d = tf.data.TFRecordDataset(input_file)
        if is_training:
            d = d.repeat()
            d = d.shuffle(buffer_size=100)

        d = d.apply(
            tf.contrib.data.map_and_batch(
                lambda record: _decode_record(record, name_to_features),
                batch_size=batch_size,
                drop_remainder=drop_remainder))

        return d

    return input_fn


def _truncate_seq_pair(tokens_a, tokens_b, max_length):
    """Truncates a sequence pair in place to the maximum length."""

    # This is a simple heuristic which will always truncate the longer sequence
    # one token at a time. This makes more sense than truncating an equal percent
    # of tokens from each, since if one sequence is very short then each token
    # that's truncated likely contains more information than a longer sequence.
    while True:
        total_length = len(tokens_a) + len(tokens_b)
        if total_length <= max_length:
            break
        if len(tokens_a) > len(tokens_b):
            tokens_a.pop()
        else:
            tokens_b.pop()

In [13]:
def input_fn_builder(features, seq_length, is_training, drop_remainder):
  """Creates an `input_fn` closure to be passed to TPUEstimator."""

  all_input_ids = []
  all_input_mask = []
  all_segment_ids = []
  all_label_ids = []

  for feature in features:
    all_input_ids.append(feature.input_ids)
    all_input_mask.append(feature.input_mask)
    all_segment_ids.append(feature.segment_ids)
    all_label_ids.append(feature.label_ids)

  def input_fn(params):
    """The actual input function."""
    batch_size = params["batch_size"]

    num_examples = len(features)

    # This is for demo purposes and does NOT scale to large data sets. We do
    # not use Dataset.from_generator() because that uses tf.py_func which is
    # not TPU compatible. The right way to load data is with TFRecordReader.
    d = tf.data.Dataset.from_tensor_slices({
        "input_ids":
            tf.constant(
                all_input_ids, shape=[num_examples, seq_length],
                dtype=tf.int32),
        "input_mask":
            tf.constant(
                all_input_mask,
                shape=[num_examples, seq_length],
                dtype=tf.int32),
        "segment_ids":
            tf.constant(
                all_segment_ids,
                shape=[num_examples, seq_length],
                dtype=tf.int32),
        "label_ids":
            tf.constant(all_label_ids, shape=[num_examples, len(LABEL_COLUMNS)], dtype=tf.int32),
    })

    if is_training:
      d = d.repeat()
      d = d.shuffle(buffer_size=100)

    d = d.batch(batch_size=batch_size, drop_remainder=drop_remainder)
    return d

  return input_fn

In [14]:
def create_output(predictions, list_of_labels, k):
    probabilities = []
    predict = []
    for (i, prediction) in enumerate(predictions):
        preds = prediction["probabilities"]
        probabilities.append(preds)
        if len(preds) == 0:
          predict.append([])
        elif len(preds) <= k:
          predict.append(preds)
        else:
          indices = np.argpartition(preds, -k)[-k:]
          predict.append([list_of_labels[i] for i in indices])
    dff = pd.DataFrame(probabilities)
    dff.columns = list_of_labels
    dff.insert(0, 'prediction',predict)
    
    return dff

def get_accuracy(df: pd.DataFrame, label_column: str, num_test_examples: int):
  correct = 0
  for idx, row in df.iterrows():
    predictions = row['prediction']
    actual_labels = row[label_column]

    for prediction in predictions:
      if prediction in actual_labels:
        correct += 1
  
  return correct / num_test_examples

In [15]:
MAX_SEQ_LENGTH = 128

# Compute train and warmup steps from batch size
# These hyperparameters are copied from this colab notebook (https://colab.sandbox.google.com/github/tensorflow/tpu/blob/master/tools/colab/bert_finetuning_with_cloud_tpus.ipynb)
BATCH_SIZE = 32
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 3.0
# Warmup is a period of time where hte learning rate 
# is small and gradually increases--usually helps training.
WARMUP_PROPORTION = 0.1
# Model configs
SAVE_CHECKPOINTS_STEPS = 500
SAVE_SUMMARY_STEPS = 100

In [16]:
data = get_data()

# get unique genres
genres = get_unique(data, 'genres')

# get unique directors
directors = get_unique(data, 'crew')

# get unique cast members
cast = get_unique(data, 'cast')

# splitting data into training and test set
train, test = train_test_split(data, test_size = 0.3)

train.reset_index(drop=True, inplace=True)
test.reset_index(drop=True, inplace=True)

train.to_csv("train_data.csv", index=False)
test.to_csv("test_data.csv", index=False)

tokenizer = get_tokenizer()

In [17]:
# for director prediction

LABEL_COLUMN = 'crew'
LABELS = directors
NUM_LABELS = len(LABELS)
OUTPUT_DIR = './Director'

if not os.path.exists(OUTPUT_DIR):
  os.makedirs(OUTPUT_DIR)

def get_list_of_train_labels():
  result = []
  for i, row in train.iterrows():
    labels = [0] * NUM_LABELS
    for item in row[LABEL_COLUMN]:
      idx = LABELS.index(item)
      labels[idx] = 1
    result.append(labels)
  return result
  
list_of_train_labels = get_list_of_train_labels()

train_inputExamples = create_examples(df=train, list_of_train_labels=list_of_train_labels, num_labels=NUM_LABELS)
test_inputExamples = create_examples(df=test, list_of_train_labels=list_of_train_labels, num_labels=NUM_LABELS, set_labels=False)

train_features = convert_examples_to_features(train_inputExamples, MAX_SEQ_LENGTH, tokenizer)
test_features = convert_examples_to_features(test_inputExamples, MAX_SEQ_LENGTH, tokenizer)

num_train_steps = int(len(train_inputExamples) / BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

train_file = os.path.join(OUTPUT_DIR, "train.tf_record")
if not os.path.exists(train_file):
  open(train_file, 'w').close()

file_based_convert_examples_to_features(train_inputExamples, MAX_SEQ_LENGTH, tokenizer, train_file)

tf.logging.info("***** Running training *****")
tf.logging.info("  Num examples = %d", len(train_inputExamples))
tf.logging.info("  Batch size = %d", BATCH_SIZE)
tf.logging.info("  Num steps = %d", num_train_steps)

train_input_fn = file_based_input_fn_builder(train_file, MAX_SEQ_LENGTH, is_training=True, drop_remainder=True, num_labels=NUM_LABELS)

run_config = tf.estimator.RunConfig(
    model_dir = OUTPUT_DIR,
    save_summary_steps = SAVE_SUMMARY_STEPS,
    keep_checkpoint_max = 1,
    save_checkpoints_steps = SAVE_CHECKPOINTS_STEPS
)

bert_config = modeling.BertConfig.from_json_file(BERT_CONFIG)

model_fn = model_fn_builder(
  bert_config=bert_config,
  num_labels= NUM_LABELS,
  init_checkpoint=BERT_INIT_CHKPNT,
  learning_rate=LEARNING_RATE,
  num_train_steps=num_train_steps,
  num_warmup_steps=num_warmup_steps,
  use_tpu=False,
  use_one_hot_embeddings=False)

estimator = tf.estimator.Estimator(
  model_fn=model_fn,
  config=run_config,
  params={"batch_size": BATCH_SIZE})


INFO:tensorflow:***** Running training *****
INFO:tensorflow:  Num examples = 3316
INFO:tensorflow:  Batch size = 32
INFO:tensorflow:  Num steps = 310
INFO:tensorflow:Using config: {'_model_dir': './Director', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 1, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f3e827b5550>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True,

In [18]:
print('Beginning Training!')
curtime = datetime.now()
estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
print("Training time elapsed: ", datetime.now() - curtime)

Beginning Training!
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Use `tf.data.experimental.map_and_batch(...)`.
Instructions for updating:
Use `tf.data.Dataset.map(map_func, num_parallel_calls)` followed by `tf.data.Dataset.batch(batch_size, drop_remainder)`. Static tf.data optimizations will take care of using the fused implementation.
Instructions for updating:
Use `tf.cast` instead.
INFO:tensorflow:Calling model_fn.



Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate sho

In [19]:

test_file = os.path.join(OUTPUT_DIR, "test.tf_record")
if not os.path.exists(test_file):
  open(test_file, 'w').close()

file_based_convert_examples_to_features(test_inputExamples, MAX_SEQ_LENGTH, tokenizer, test_file)

predict_input_fn = file_based_input_fn_builder(
    input_file=test_file, 
    seq_length=MAX_SEQ_LENGTH, 
    is_training=False,
    drop_remainder=False,
    num_labels=NUM_LABELS)

print('Beginning predictions!')
curtime = datetime.now()
predictions = estimator.predict(predict_input_fn)
print("Predictioin time elapsed: ", datetime.now() - curtime)



Beginning predictions!
Predictioin time elapsed:  0:00:00.000359


In [20]:
output_df = create_output(predictions, LABELS, 4)

output_df['id'] = np.asarray(test['id'])
result_df = pd.merge(test, output_df, on='id', how='inner')
result_df.to_csv("result.csv", index=False)
print(f'accuracy of prediction {LABEL_COLUMN} is {get_accuracy(result_df, LABEL_COLUMN, len(test)):.9f}')

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:num_labels:2542;logits:Tensor("loss/BiasAdd:0", shape=(?, 2542), dtype=float32);labels:Tensor("loss/Cast:0", shape=(?, 2542), dtype=float32)
INFO:tensorflow:**** Trainable Variables ****
mode: infer probabilities: Tensor("loss/Sigmoid:0", shape=(?, 2542), dtype=float32)
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./Director/model.ckpt-310
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
accuracy of prediction crew is 0.001406470
